#### **Generación de archivo farmacias_departamento.csv**
El archivo de farmacias por departamento es utilizado para un gráfico de barras del dashboard

In [3]:
import pandas as pd

# Carga archivo csv de farmacias, cada farmacia puede tener varias filas dependiendo
# de las zonas que quedan comprendidas en un radio de 1 km (buffer)
data = pd.read_csv('data/interseccion_zonas_buffer.csv')
df = data.copy()

# Convierte CODCOMP a string con 0 a la izquierda si departamento <= 9 
df['CODCOMP_STR'] = (df['CODCOMP'].astype(str)).str.zfill(10)

# Convierte CODLOC a string con 0 a la izquierda si departamento <= 9 
df['CODLOC_STR'] = (df['CODLOC'].astype(str)).str.zfill(5)

# Elimina las columnas innecesarias
df.drop(columns=['fid', 'NOMLOC', 'CODCOMP_A', 'CODCOMP', 'CODLOC'], inplace=True)

# Se crea el dataframe df_farmacias_depto a partir de las columnas NOMBDEPTO, NOMBRE, DIRECCION
df_farmacias_depto = df[['NOMBDEPTO', 'NOMBRE', 'DIRECCION']]
# Crea la columna CANT_FARMACIAS con la concatenación de NOMBRE y DIRECCION de la farmacia
df_farmacias_depto['CANT_FARMACIAS'] = df_farmacias_depto['NOMBRE'] + df_farmacias_depto['DIRECCION']
df_farmacias_depto.drop(columns=['NOMBRE', 'DIRECCION'], inplace=True)
# Elimina filas duplicadas
df_farmacias_depto = df_farmacias_depto.drop_duplicates()
# Cuenta farmacias por nombre de departamento
df_farmacias_depto = df_farmacias_depto.groupby(by = ['NOMBDEPTO']).count()

# Crea el archivo farmacias_departamento.csv
df_farmacias_depto.to_csv('data/farmacias_departamento.csv')
print(df_farmacias_depto)

C:\Users\cchabay\AppData\Local\Temp\ipykernel_13924\3439828179.py:5: DtypeWarning: Columns (8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data/interseccion_zonas_buffer.csv')
C:\Users\cchabay\AppData\Local\Temp\ipykernel_13924\3439828179.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_farmacias_depto['CANT_FARMACIAS'] = df_farmacias_depto['NOMBRE'] + df_farmacias_depto['DIRECCION']
C:\Users\cchabay\AppData\Local\Temp\ipykernel_13924\3439828179.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df

                CANT_FARMACIAS
NOMBDEPTO                     
ARTIGAS                     12
CANELONES                   72
CERRO LARGO                  9
COLONIA                     36
DURAZNO                     11
FLORES                       9
FLORIDA                     15
LAVALLEJA                   13
MALDONADO                   48
MONTEVIDEO                 172
PAYSANDU                    15
RIO NEGRO                   16
RIVERA                      12
ROCHA                       19
SALTO                       12
SAN JOSE                    20
SORIANO                     21
TACUAREMBO                  23
TREINTA Y TRES               9


#### **Generación de los siguientes archivos:**
- loc_pob_farm.csv - Población y cantidad de farmacias por localidad
- loc_grandes_sin_farm.csv - Localidades de mas de 3.000 habitantes sin farmacias a menos de 1 km de distancia
- loc_chicas_con_farm.csv - Localidades de menos de 3.000 habitantes con farmacias a menos de 1 km de distancia

In [4]:
# Se crea el dataframe df_localidad_poblacion con poblacion x localidad

df_localidad_poblacion = df[['NOMBDEPTO', 'CODCOMP_STR', 'CODLOC_STR', 'NOMBLOC', 'POBLACION']]
df_localidad_poblacion = df_localidad_poblacion.drop_duplicates()

# Elimina las columnas innecesarias
df_localidad_poblacion.drop(columns=['CODCOMP_STR'], inplace=True)
# Agrupa por localidad sumando población
df_localidad_poblacion = df_localidad_poblacion.groupby(by = ['NOMBDEPTO', 'CODLOC_STR', 'NOMBLOC']).sum()
# Ordena por localidad
df_localidad_poblacion = df_localidad_poblacion.sort_values(by='CODLOC_STR')
df_localidad_poblacion = df_localidad_poblacion.reset_index()

# Se crea el dataframe df_localidad_farmacias

df_localidad_farmacias = df[['CODLOC_STR', 'NOMBRE', 'DIRECCION']]
# Crea la columna CANT_FARMACIAS con la concatenación de NOMBRE y DIRECCION de la farmacia
df_localidad_farmacias['CANT_FARMACIAS'] = df_localidad_farmacias['NOMBRE'] + df_localidad_farmacias['DIRECCION']
# Elimina columnas innecesarias
df_localidad_farmacias.drop(columns=['NOMBRE', 'DIRECCION'], inplace=True)
# Elimina filas duplicadas
df_localidad_farmacias = df_localidad_farmacias.drop_duplicates()
# Agrupa por localidad contando farmacias
df_localidad_farmacias = df_localidad_farmacias.groupby(by = ['CODLOC_STR']).count()
df_localidad_farmacias = df_localidad_farmacias.sort_values(by='CODLOC_STR')
df_localidad_farmacias = df_localidad_farmacias.reset_index()
df_localidad_farmacias.drop(columns=['CODLOC_STR'], inplace=True)

# Crea el dataframe df_loc_pob_farm con las columnas de los
# dataframes df_localidad_poblacion y df_localidad_farmacias
df_loc_pob_farm = pd.concat([df_localidad_poblacion, df_localidad_farmacias], axis = 1)

# Crea dataframe con POBLACION > 3000 y CANT_FARMACIAS == 0'
df_loc_pob_farm.query('POBLACION > 3000 and CANT_FARMACIAS == 0').to_csv('data/loc_grandes_sin_farm.csv',
                                                                        columns=['NOMBDEPTO','NOMBLOC','POBLACION','CANT_FARMACIAS'], index=False)
# Crea dataframe con POBLACION < 3000 y CANT_FARMACIAS > 0
df_loc_pob_farm.query('POBLACION < 3000 and CANT_FARMACIAS != 0').to_csv('data/loc_chicas_con_farm.csv',
                                                                        columns=['NOMBDEPTO','NOMBLOC','POBLACION','CANT_FARMACIAS'], index=False)

# Crea el archivo df_loc_pob_farm.csv
df_loc_pob_farm.to_csv('data/loc_pob_farm.csv', columns=['NOMBDEPTO','NOMBLOC','POBLACION','CANT_FARMACIAS'], index=False)


C:\Users\cchabay\AppData\Local\Temp\ipykernel_13924\2868486144.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_localidad_farmacias['CANT_FARMACIAS'] = df_localidad_farmacias['NOMBRE'] + df_localidad_farmacias['DIRECCION']
C:\Users\cchabay\AppData\Local\Temp\ipykernel_13924\2868486144.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_localidad_farmacias.drop(columns=['NOMBRE', 'DIRECCION'], inplace=True)
